In [1]:
# ================================================
# Preprocessing All Gene Expression Datasets
# ================================================
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

# -----------------------------------------------
# 1. Define paths
# -----------------------------------------------
base_dir = Path("../data")
processed_dir = base_dir / "processed"
cleaned_dir = base_dir / "cleaned"
cleaned_dir.mkdir(parents=True, exist_ok=True)

datasets = {
    "breast": processed_dir / "breast_processed.csv",
    "ovarian": processed_dir / "ovarian_processed.csv",
    "lung": processed_dir / "lung_processed.csv"
}

# -----------------------------------------------
# 2. Define helper function
# -----------------------------------------------
def preprocess_dataset(file_path, disease_name):
    print(f"\n🔹 Preprocessing {disease_name} dataset...")

    # Load dataset
    df = pd.read_csv(file_path)
    print(f"Initial shape: {df.shape}")

    # Drop duplicate genes (if any)
    df = df.drop_duplicates(subset=["Gene"], keep="first")

    # Drop rows with too many missing values
    df = df.dropna(thresh=int(df.shape[1] * 0.8))

    # Replace remaining NaNs with column means
    df = df.fillna(df.mean(numeric_only=True))

    # Normalize numeric columns (gene expression values)
    numeric_df = df.select_dtypes(include=[np.number])
    scaler = MinMaxScaler()
    df[numeric_df.columns] = scaler.fit_transform(numeric_df)

    # Save cleaned dataset
    out_path = cleaned_dir / f"{disease_name}_cleaned.csv"
    df.to_csv(out_path, index=False)
    print(f"✅ Saved cleaned dataset: {out_path} (shape: {df.shape})")

# -----------------------------------------------
# 3. Run preprocessing for all datasets
# -----------------------------------------------
for disease, path in datasets.items():
    preprocess_dataset(path, disease)

print("\n🎉 All datasets preprocessed and saved in data/cleaned/")



🔹 Preprocessing breast dataset...
Initial shape: (13299, 87)
✅ Saved cleaned dataset: ..\data\cleaned\breast_cleaned.csv (shape: (13299, 87))

🔹 Preprocessing ovarian dataset...
Initial shape: (22189, 64)
✅ Saved cleaned dataset: ..\data\cleaned\ovarian_cleaned.csv (shape: (22189, 64))

🔹 Preprocessing lung dataset...
Initial shape: (22189, 247)
✅ Saved cleaned dataset: ..\data\cleaned\lung_cleaned.csv (shape: (22189, 247))

🎉 All datasets preprocessed and saved in data/cleaned/


In [10]:
import pandas as pd
import re
from pathlib import Path

# Paths
data_dir = Path("../data")
mapped_dir = data_dir / "mapped"
cleaned_dir = data_dir / "cleaned"
labeled_dir = data_dir / "labeled"
labeled_dir.mkdir(parents=True, exist_ok=True)

datasets = {
    "breast": data_dir / "GSE15852_series_matrix.txt",
    "ovarian": data_dir / "GSE18520_series_matrix.txt",
    "lung": data_dir / "GSE31210_series_matrix.txt"
}

processed_dir = data_dir / "preprocessed"  # 👈 change directory here

processed_files = {
    "breast": processed_dir / "breast_cleaned.csv",
    "ovarian": processed_dir / "ovarian_cleaned.csv",
    "lung": processed_dir / "lung_cleaned.csv"
}




def extract_labels(gse_file):
    """
    Extracts binary labels (0=Normal, 1=Cancer) for each sample
    from GEO meta info like "Normal BC0043N" "Cancer BC0043T".
    """
    with open(gse_file, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    # Get all sample-related metadata lines
    sample_lines = [l.strip() for l in lines if l.startswith("!Sample_title") or "Sample_title" in l]
    
    if not sample_lines:
        raise ValueError("❌ No sample title lines found in metadata.")
    
    # Merge and split on quotes to isolate each sample title
    text = " ".join(sample_lines)
    samples = re.findall(r'"([^"]+)"', text)
    
    if not samples:
        raise ValueError("❌ No samples found within quotes in metadata.")

    print(f"🧬 Found {len(samples)} samples in metadata.")
    
    # Determine label (1 for cancer/tumor, 0 for normal/control)
    labels = []
    for s in samples:
        s_lower = s.lower()
        if "cancer" in s_lower or "tumor" in s_lower or "malignant" in s_lower:
            labels.append(1)
        else:
            labels.append(0)
    
    print(f"✅ Labels extracted: {sum(labels)} cancer / {len(labels) - sum(labels)} normal.")
    return labels



for disease, gse_file in datasets.items():
    print(f"\n🔹 Adding labels for {disease} dataset...")
    try:
        labels = extract_labels(gse_file)
    except Exception as e:
        print(f"❌ Error extracting labels for {disease}: {e}")
        continue

    df = pd.read_csv(processed_files[disease])

    # Transpose (genes as columns → samples as rows)
    df_t = df.set_index("Gene").T

    # Check matching length
    if len(labels) != df_t.shape[0]:
        print(f"⚠️ Label/sample mismatch: {len(labels)} labels vs {df_t.shape[0]} samples. Adjusting to min length.")
        min_len = min(len(labels), df_t.shape[0])
        labels = labels[:min_len]
        df_t = df_t.iloc[:min_len]

    df_t["status"] = labels
    out_path = labeled_dir / f"{disease}_labeled.csv"
    df_t.to_csv(out_path, index=False)
    print(f"✅ Saved labeled dataset: {out_path}")



🔹 Adding labels for breast dataset...
🧬 Found 86 samples in metadata.
✅ Labels extracted: 43 cancer / 43 normal.
✅ Saved labeled dataset: ..\data\labeled\breast_labeled.csv

🔹 Adding labels for ovarian dataset...
🧬 Found 63 samples in metadata.
✅ Labels extracted: 53 cancer / 10 normal.
✅ Saved labeled dataset: ..\data\labeled\ovarian_labeled.csv

🔹 Adding labels for lung dataset...
🧬 Found 246 samples in metadata.
✅ Labels extracted: 0 cancer / 246 normal.
✅ Saved labeled dataset: ..\data\labeled\lung_labeled.csv


In [13]:
import pandas as pd
import re
from pathlib import Path

# ----------------------------
# Define paths
# ----------------------------
data_dir = Path("../data")
gse_file = data_dir / "GSE31210_series_matrix.txt"
processed_file = data_dir / "preprocessed" / "lung_cleaned.csv"
labeled_dir = data_dir / "labeled"
labeled_dir.mkdir(parents=True, exist_ok=True)
out_path = labeled_dir / "lung_labeled.csv"


# ----------------------------
# Extract labels specifically from "tissue:" entries
# ----------------------------
def extract_labels_lung(gse_file):
    """
    Extracts binary labels (1=cancer/tumor/adenocarcinoma, 0=normal)
    for the lung dataset (GSE31210).
    Only considers 'tissue:' metadata lines.
    """
    with open(gse_file, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    # ✅ Filter only lines describing tissue info
    tissue_lines = [
        l.strip() for l in lines
        if l.startswith("!Sample_characteristics_ch1") and "tissue:" in l.lower()
    ]

    # Extract each "tissue: ..." entry
    text = " ".join(tissue_lines)
    tissues = re.findall(r'"tissue: ([^"]+)"', text)

    if not tissues:
        raise ValueError("❌ No 'tissue:' entries found in GEO file.")
    print(f"🧬 Found {len(tissues)} tissue samples in metadata.")

    # Convert to binary labels
    labels = []
    for t in tissues:
        t_lower = t.lower()
        if any(word in t_lower for word in ["tumor", "cancer", "adenocarcinoma", "carcinoma", "nsclc"]):
            labels.append(1)
        elif "normal" in t_lower:
            labels.append(0)
        else:
            print(f"⚠️ Unknown tissue type: {t}")
            labels.append(None)

    print(f"✅ Labels extracted: {sum(l == 1 for l in labels)} cancer / {sum(l == 0 for l in labels)} normal.")
    return labels


# ----------------------------
# Apply labels to dataset
# ----------------------------
labels = extract_labels_lung(gse_file)
df = pd.read_csv(processed_file)

# Transpose (genes as columns → samples as rows)
df_t = df.set_index("Gene").T

# Align label count and sample count
min_len = min(len(labels), df_t.shape[0])
df_t = df_t.iloc[:min_len]
labels = labels[:min_len]

df_t["status"] = labels

# Save labeled dataset
df_t.to_csv(out_path, index=False)
print(f"✅ Saved labeled lung dataset: {out_path}")

# Display summary
print("\n📊 Dataset Summary:")
print(f"Samples: {df_t.shape[0]}")
print(f"Genes: {df_t.shape[1] - 1}")
print(df_t['status'].value_counts())


🧬 Found 246 tissue samples in metadata.
✅ Labels extracted: 226 cancer / 20 normal.
✅ Saved labeled lung dataset: ..\data\labeled\lung_labeled.csv

📊 Dataset Summary:
Samples: 246
Genes: 22189
status
1    226
0     20
Name: count, dtype: int64


In [3]:
import pandas as pd

# Load datasets
df_processed = pd.read_csv("../data/preprocessed/breast_cleaned.csv")   # genes as rows, samples as columns
df_labeled = pd.read_csv("../data/labeled/breast_labeled.csv")         # samples as rows, with status


In [4]:
# Extract status values from the labeled file
status_values = df_labeled["status"].values

# Get sample names (columns in processed file)
sample_names = df_labeled.columns[:-1]  # all columns except 'status'


In [5]:
# Create status DataFrame with same column order as processed dataset
status_row = pd.DataFrame([status_values], columns=sample_names)
status_row.index = ["status"]

# Append the status row at the bottom
df_with_status = pd.concat([df_processed, status_row])


ValueError: 13299 columns passed, passed data had 1 columns

In [6]:
print("Processed shape:", df_processed.shape)
print("Labeled shape:", df_labeled.shape)


Processed shape: (13299, 87)
Labeled shape: (1, 13300)


In [6]:
import json

path = "../models/breast_genes.json"

with open(path, "r") as f:
    genes = json.load(f)

expanded = []

for g in genes:
    g = g.strip()
    
    # Split genes like "ABC1///ABC2"
    if "///" in g:
        expanded.extend([p.strip() for p in g.split("///") if p.strip()])
    else:
        expanded.append(g)

# Remove duplicates while preserving order
clean = list(dict.fromkeys(expanded))

with open(path, "w") as f:
    json.dump(clean, f, indent=2)

print("Cleaning complete.")
print("Before:", len(genes))
print("After:", len(clean))


Cleaning complete.
Before: 13299
After: 13909


In [7]:
import json

path = "../models/lung_genes.json"

with open(path, "r") as f:
    genes = json.load(f)

expanded = []

for g in genes:
    g = g.strip()
    
    # Split genes like "ABC1///ABC2"
    if "///" in g:
        expanded.extend([p.strip() for p in g.split("///") if p.strip()])
    else:
        expanded.append(g)

# Remove duplicates while preserving order
clean = list(dict.fromkeys(expanded))

with open(path, "w") as f:
    json.dump(clean, f, indent=2)

print("Cleaning complete.")
print("Before:", len(genes))
print("After:", len(clean))


Cleaning complete.
Before: 22189
After: 22836


In [8]:
import json

path = "../models/ovarian_genes.json"

with open(path, "r") as f:
    genes = json.load(f)

expanded = []

for g in genes:
    g = g.strip()
    
    # Split genes like "ABC1///ABC2"
    if "///" in g:
        expanded.extend([p.strip() for p in g.split("///") if p.strip()])
    else:
        expanded.append(g)

# Remove duplicates while preserving order
clean = list(dict.fromkeys(expanded))

with open(path, "w") as f:
    json.dump(clean, f, indent=2)

print("Cleaning complete.")
print("Before:", len(genes))
print("After:", len(clean))


Cleaning complete.
Before: 22189
After: 22836
